In [31]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [32]:
# data_path=r'D:\PythonDev\MyQuantFinProject\Data\thai-small_2023-now.csv'
data_path=r'D:\PythonDev\MyQuantFinProject\Data\thai-active_2023-now.csv'

#trade_sheet='ThaiNov22-Now23'
trade_sheet='TH_260623-Now'
file_name=trade_sheet
file_name=f'ACT_{trade_sheet}'
noFee=False


start_date='2023-06-26'
end_date='2023-12-31'

fee_sheet='Fee'
# data_path=r'D:\PythonDev\MyQuantFinProject\Data\thai_fund.csv'
trade_plan_path='data/Config-BuySellTrade.xlsx'

# excludeFund=['ASP-T12','KKP-TQG']
excludeFund=[]

TotalBudget=100*3000



if noFee==True:
 report_path=f'report/{file_name}(no-fee).xlsx'
else:
 report_path=f'report/{file_name}.xlsx' 
print(report_path)
# trade_sheet='BuySell_Buy_Hold'
# report_path=f'report/BuySell_4ETFFunds.xlsx'


report/ACT_TH_260623-Now.xlsx


# Read Price CSV

In [33]:
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
df.columns=['symbol','date','price']
df=df.query('date>=@start_date and date<=@end_date')
df.info()
print(df.sample(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 116 to 1140
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  329 non-null    object        
 1   date    329 non-null    datetime64[ns]
 2   price   329 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 10.3+ KB
            symbol       date  price
947     TISCOFLEXP 2023-07-18  53.11
956     TISCOFLEXP 2023-08-02  53.13
789      KFDYNAMIC 2023-07-25  37.89
641   KKP-ACT-EQ-A 2023-08-18  16.50
155        ASP-T12 2023-08-23  11.84
1121         TSF-A 2023-08-04  73.50
126        ASP-T12 2023-07-10  11.08
468        K-SET50 2023-08-03  33.49
631   KKP-ACT-EQ-A 2023-08-03  16.48
289       ASP-THEQ 2023-07-10  14.56


# List Symbol and Plot Chart

In [34]:
list_symbols=list(df['symbol'].unique())
list_symbols=[ f for f in list_symbols if f not in excludeFund ]
list_symbols
# list_symbols=['ACWI']


['ASP-T12',
 'ASP-THEQ',
 'K-SET50',
 'KKP-ACT-EQ-A',
 'KFDYNAMIC',
 'TISCOFLEXP',
 'TSF-A']

In [35]:
# print(list_symbols)
# for  symbol in list_symbols:
#     print(f"=============================={symbol}==============================")
#     dfFund=df.query('symbol==@symbol')
#     plt.figure(figsize=(15, 8))
#     plt.plot(dfFund['date'],dfFund['price'])
#     plt.show()

# Get Trade Data and Fee

In [36]:
# read feeon 
dfAssetInfo=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=fee_sheet,\
                    index_col='symbol', dtype ={'buy_fee_pct': np.float64,'sell_fee_pct': np.float64})

# check where  fund fee  in portfolio  are avaible?
dfNoFeeInfo= [ item for item in list_symbols if item not in list(dfAssetInfo.index)]
dfAssetInfo=dfAssetInfo.loc[list(list_symbols),:]

if noFee==True:
    dfAssetInfo[['buy_fee_pct','sell_fee_pct']]=0
    
print("fee info")
print(dfAssetInfo) 
# For instance   AAXJ Price=100 but we need to pay 101 $

fee info
              buy_fee_pct  sell_fee_pct region
symbol                                        
ASP-T12              1.25           0.0   thai
ASP-THEQ             1.00           0.0   thai
K-SET50              0.00           0.1   thai
KKP-ACT-EQ-A         0.50           0.0   thai
KFDYNAMIC            0.50           0.5   thai
TISCOFLEXP           1.00           0.0   thai
TSF-A                1.05           0.0   thai


In [37]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name=trade_sheet, \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})

#total buy portion is equal or greater than sell portion
buy_sum_act=dfPlan.query("action=='b'").portion_pct.sum()
sell_sum_act=dfPlan.query("action=='s'").portion_pct.sum()

all_portion=0

In [38]:
if buy_sum_act<sell_sum_act:
  raise Exception("Not allow Buy <= Sell")   
else:
    if buy_sum_act>sell_sum_act:
       print("=============Calculate the last portion to be sellable")
       sellAllAtNow=buy_sum_act-sell_sum_act 
       dfLastPrice=pd.DataFrame(columns=list(dfPlan.columns), data=[ [df['date'].max(),sellAllAtNow,'s']])
       print(dfLastPrice)
       dfPlan=pd.concat([dfPlan,dfLastPrice])
       dfPlan=dfPlan.reset_index(drop=True)

    tradeDateList= [ d for d in list(dfPlan['trade_date']) ]    
    all_portion=buy_sum_act   
    print(f"portion = {all_portion} on")
    
print("==========================Summarize All Trade==========================================")
print(f"No Buy/Sell {dfPlan.groupby(['action']).size()}")
print(dfPlan)

=============Calculate the last portion to be sellable
  trade_date  portion_pct action
0 2023-09-01        100.0      s
portion = 100.0 on
==========================Summarize All Trade==========================================
No Buy/Sell action
b    5
s    1
dtype: int64
  trade_date  portion_pct action
0 2023-06-30         20.0      b
1 2023-07-14         20.0      b
2 2023-07-20         20.0      b
3 2023-08-10         20.0      b
4 2023-08-22         20.0      b
5 2023-09-01        100.0      s


In [39]:
# To make it easier for use to understand portfolio transatoin 
rrr=TotalBudget % all_portion
if rrr !=0 :
  raise Exception(f"Your Budget {TotalBudget} is  not proportional  Portion with {all_portion}")
else:
  print(f"Your Budget {TotalBudget} is  proportional  Portion with {all_portion}")

Your Budget 300000 is  proportional  Portion with 100.0


# Custom Function

In [40]:
# figure out the amount of qty from total fund total value
def get_qty_buy(item):
    xCost=(item['portion_pct']/all_portion)*TotalBudget  
    xPrice=item['price_on_fee']
    qty=xCost/xPrice
    return qty

# figure the amount of  qty out  from total qty
def get_qty_sell(item,TotalQty):  
    qty=(item['portion_pct']/all_portion)*TotalQty
    return qty

In [41]:
def set_fee_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    fee_type='buy_fee_pct'
  elif  item['action']=='s':
    fee_type='sell_fee_pct'
  try:
     return dfAssetInfo.loc[x,fee_type]
  except Exception as ex:
     print(f"Error : no {x} to get fund fee")
     return 0

In [42]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])

dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL


In [43]:
#https://www.w3resource.com/pandas/dataframe/dataframe-apply.php

for symbol in list_symbols:
    print(f"*******************{symbol}*******************")

    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    # if len(tradeDateList)==len(dfBS):
        
     # Step1  join price data (filter by sybom and the date whiich you have buy/sell action)   
    dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')
     # remove  redundant data  
    dfBS=dfBS.drop(columns=['trade_date'])     
       
    
     # Step2 calculate fee function
    dfBS['fee']=dfBS.apply(set_fee_by_action,axis=1) # 2.1
    
     # The price that allows you to buy the fund is more expensive than the actual asset price  1%  ,other the other had 
     # Buy price*1.0x=??   
    dfBS['price_on_fee']=dfBS.apply(lambda item : item['price']*(1+ (item['fee']/100)) if item['action']=='b' else item['price']*(1-(item['fee']/100)), axis=1)
    dfBS['price_on_fee']=dfBS['price_on_fee'].round(2) #2.2   
    # print(dfBS)
        
     # Step3  To make it easier to calculate and trading sumary in tearm of qty both buy/sell , both use different way
     # we recommend spliting this dataframe  into 2 dataframe
     #  BuyDataFrame and Sell Date , and we will merge both later  
    
    #3.1 query only buy action
    print("======================Buy=======================")
    buyDF=dfBS.query("action=='b'")   
    
    # 3.2 send the  row to function to calculate qty 
    buyDF['qty']=buyDF.apply(get_qty_buy,axis=1) 
    buyDF['value']=round(buyDF['qty']*buyDF['price_on_fee'],0)
    
    
    # 1 is buy (I will take columne to apply cumsum function to calcaulate cumulative return
    buyDF['action_value']=buyDF['value']
    buyDF['action_qty']=buyDF['qty']
    
    # 3.3 Total qty and value and avg
    TotalQty=buyDF['qty'].sum()
    TotalValue=round(buyDF['value'].sum(),0)
    AvgPrice=round(TotalValue/TotalQty,2)
    
    # print(buyDF[['date','portion_pct','value' ,'price','fee','price_on_fee','qty']])
    
    
    # Step4 follow every step corresponidng to prev step#3
    print("=======================Sell=======================")
    # step#4.1
    sellDF=dfBS.query("action=='s'")  
    
    # ste#4.2
    sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
    sellDF['value']=round(sellDF['qty']*sellDF['price_on_fee'],0)
    
    # -1 is sell (I will take columne to apply cumsum function to calcaulate cumulative return
    sellDF['action_value']=sellDF['value']*-1
    sellDF['action_qty']=sellDF['qty']*-1
    
    # print(sellDF[['date' ,'portion_pct','value','price','fee','price_on_fee','qty']])
    

    
    print("======================Merge Buy & Sell======================")
    # step 5  concatenate buy and sell dataframe 
    dfTran=pd.concat([buyDF,sellDF],axis=0)
    dfTran=dfTran.sort_values(by=['date'])
    dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)

    # step 6 accumulate value and qury over entire data since start buying to last record 
    dfTran['net_value']=dfTran['action_value'].cumsum(axis = 0) 
    dfTran['net_qty']=dfTran['action_qty'].cumsum(axis = 0)  
    dfTran['avg_price']=round( dfTran['net_value']/dfTran['net_qty'],2)
    
    dfTran=dfTran.sort_values(by=['date'])
    # print(dfTran[['date' ,'action','price_on_fee','qty','net_qty','value','net_value','avg_price']])
    print(dfTran[['date' ,'action','price_on_fee','qty','value','avg_price']])
    
    dict_dfTran[symbol]=dfTran
    
    print("=======================Summarize Gain/Loss=======================")
    #step 7 create total buy/sell value , avg prie and profit/loss(value and %) of  all funds 
    # step 7.1 apply sum agg over  value and qty to find avg throughout  the whold dataframe
    SellTotalQty=sellDF['qty'].sum()
    SellTotalValue=round(sellDF['value'].sum(),0)
    SellAvgPrice=round(SellTotalValue/SellTotalQty,0)
    
    print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")
    print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")
    

    
    # step #7.2
    PL=round(SellTotalValue-TotalValue,0)
    PCT_PL=round(PL/TotalValue*100,2)
    
    # step# 7.3 
    # you take amont 100000.0 and gain profit103500 from investment on this etf fund 
    summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,\
                  'AvgSellPrice':SellAvgPrice, 'PL':PL,'Percent_PL':PCT_PL
                 }
    dfSummary=dfSummary.append(summary_data,ignore_index=True)
    
    print(dfSummary)

    


*******************ASP-T12*******************
======================Buy=======================
=======================Sell=======================
======================Merge Buy & Sell======================
        date action  price_on_fee           qty     value  avg_price
0 2023-06-30      b         11.32   5300.353357   60000.0      11.32
1 2023-07-14      b         11.50   5217.391304   60000.0      11.41
2 2023-07-20      b         11.59   5176.876618   60000.0      11.47
3 2023-08-10      b         11.74   5110.732538   60000.0      11.54
4 2023-08-22      b         11.93   5029.337804   60000.0      11.61
5 2023-09-01      s         11.72  25834.691621  302783.0       -inf
=======================Summarize Gain/Loss=======================
Total Buy ASP-T12 : qty=25834.691621198548 and value=300000.0 at AvgPrice=11.61
Total Sell ASP-T12 : qty=25834.691621198548 and value=302783.0 at AvgPrice=12.0
    Symbol  BuyValue  AvgBuyPrice  SellValue  AvgSellPrice      PL  Percent_PL
0  AS

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_1796\186035005.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_1796\186035005.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=round(buyDF['qty']*buyDF['price_on_fee'],0)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_1796\186035005.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

        date action  price_on_fee           qty     value  avg_price
0 2023-06-30      b         16.70   3592.814371   60000.0      16.70
1 2023-07-14      b         16.54   3627.569528   60000.0      16.62
2 2023-07-20      b         16.49   3638.568830   60000.0      16.58
3 2023-08-10      b         16.71   3590.664273   60000.0      16.61
4 2023-08-22      b         16.92   3546.099291   60000.0      16.67
5 2023-09-01      s         17.15  17995.716293  308627.0       -inf
=======================Summarize Gain/Loss=======================
Total Buy KKP-ACT-EQ-A : qty=17995.716292937766 and value=300000.0 at AvgPrice=16.67
Total Sell KKP-ACT-EQ-A : qty=17995.716292937766 and value=308627.0 at AvgPrice=17.0
         Symbol  BuyValue  AvgBuyPrice  SellValue  AvgSellPrice      PL  \
0       ASP-T12  300000.0        11.61   302783.0          12.0  2783.0   
1      ASP-THEQ  300000.0        15.02   307069.0          15.0  7069.0   
2       K-SET50  300000.0        33.26   305902.0       

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_1796\186035005.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfSummary=dfSummary.append(summary_data,ignore_index=True)


In [44]:
dfSummary

,Symbol,BuyValue,AvgBuyPrice,SellValue,AvgSellPrice,PL,Percent_PL
0,ASP-T12,300000.0,11.61,302783.0,12.0,2783.0,0.93
1,ASP-THEQ,300000.0,15.02,307069.0,15.0,7069.0,2.36
2,K-SET50,300000.0,33.26,305902.0,34.0,5902.0,1.97
3,KKP-ACT-EQ-A,300000.0,16.67,308627.0,17.0,8627.0,2.88
4,KFDYNAMIC,300000.0,38.53,305880.0,39.0,5880.0,1.96
5,TISCOFLEXP,300000.0,54.10,316109.0,57.0,16109.0,5.37
6,TSF-A,300000.0,75.79,303860.0,77.0,3860.0,1.29


In [45]:
dfPort=dfSummary[['BuyValue','SellValue','PL']].sum().to_frame('All Funds')
dfPort=dfPort.T
dfPort['PercentPL']=round( (dfPort['PL']/dfPort['BuyValue'] *100) ,0)
dfPort

,BuyValue,SellValue,PL,PercentPL
All Funds,2100000.0,2150230.0,50230.0,2.0


In [46]:
writer=pd.ExcelWriter(report_path,engine='xlsxwriter') 

dfPort.to_excel(writer,sheet_name='TotalPort',index=False)

dfSummary=dfSummary.sort_values(by=['Percent_PL'],ascending=False)
dfSummary.to_excel(writer, sheet_name="Summary",index=False)

for symbol,tranDF in dict_dfTran.items():
 tranDF=tranDF.drop(columns=['symbol'])   
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
writer.save()

writer.close()

print("Successfully")

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_1796\886247830.py:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Successfully


C:\Users\pongthsa\AppData\Roaming\Python\Python39\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
